In [ ]:
import numpy as np
import cvxopt as cvx
import matplotlib
import matplotlib.cm
import matplotlib.pyplot as plt
from numpy import log,exp
%matplotlib inline 
matplotlib.rcParams['figure.figsize'] = (10, 6)

### Let us generate some data:

In [ ]:
np.random.seed(420) #makes data reproducible

Ides = 0.1
pmax = 2.
N = 30
M = 10
slope = 0.6

xxx = [-0.5]
yyy = [0.]
x = []
y = []
for i in range(N):
    s = np.random.randn()
    yyy.append(yyy[-1]+slope * s)
    xxx.append(xxx[-1]+1.)
    x.append(0.5*(xxx[-1]+xxx[-2]))
    y.append(0.5*(yyy[-1]+yyy[-2]))
    
ymax = max(yyy)    

ylamps = [ymax +2.] * M
xlamps = sorted(np.random.rand(M)*N)

plt.plot(xxx,yyy)
plt.scatter(x,y,color = 'red')
plt.scatter(xlamps,ylamps, color = 'green')

The blue profile is a collection of $N=30$ patches. The green dots above are lamps. The goal of this exercize is to set the power of each lamp, in order to obtain an intensity *as uniform as possible* on all patches. The quantity of interest is in fact $\log I$. We propose to minimize
$$\max_{k=1,\ldots,N}\ | \log I_k - \log I_{des} | ,$$
subject to the constraint that the power of each lamp must be in the interval $[0,p_{max}]$.

Physics tell us that the intesity on the $i$th patch is given by
$ I_i = \sum_{j=1}^M A_{i,j}\, p_j$,
where $p_j$ is the power of the $j$th lamp and
$$A_{i,j} = \left\{\begin{array}{cl}
                    \frac{1}{(r_{i,j})^2} \cos \theta_{i,j} & \textrm{if patch $i$ receives light from lamp $j$}\\
                    0 & \textrm{otherwise.}
                    \end{array} \right.                    
$$
In this formula, $r_{i,j}$ denotes the distance from patch $i$ to lamp $j$, and $\theta_{i,j}$ is the illumination angle.

--

### Computation of Matrix $A$

In [ ]:
#r[i,j] is the distance from lamp j to patch i
r = np.zeros((N,M))
for i in range(N):
    for j in range(M):
        r[i,j] = np.linalg.norm([x[i]-xlamps[j],y[i]-ylamps[j]])


#theta[i,j] is illumination angle from lamp j to patch i
costheta = np.zeros((N,M))
for i in range(N):
    for j in range(M):
        #index of xxx_k's between xi and xlamp_j
        index_x_between = [k for k in range(N+1) if np.sign((xxx[k]-x[i])*(xxx[k]-xlamps[j])) == -1]
        y_patch = [yyy[k] for k in index_x_between]
        y_line_from_lamp = [ymax +2. + (xxx[k]-xlamps[j]) * (y[i]-ymax-2.)/(x[i]-xlamps[j]) for k in index_x_between]

        if any([y2<=y1 for y1,y2 in zip(y_patch,y_line_from_lamp)]):
            #this patch gets no line from this lamp (it is hidden by some other patch)
            costheta[i,j] = 0.
        else:
            dy = yyy[i+1]-yyy[i]
            normal_dir = np.array([-dy,1.])
            normal_dir /= np.linalg.norm(normal_dir)
            lamp_dir = np.array([xlamps[j]-x[i],ylamps[j]-y[i]])
            costheta[i,j] = (normal_dir.dot(lamp_dir))/np.linalg.norm(lamp_dir)
            
assert np.min(costheta)>=0.
assert np.min(costheta)<=1.

A = cvx.matrix(0.,(N,M))
for i in range(N):
    for j in range(M):
        A[i,j] = 1./r[i,j]**2 * costheta[i,j]

### short function to visualize a solution

In [ ]:
def visualize_sol(pp,normalized=False):
    colors = (np.log(A*pp)-np.log(Ides)).ravel()
    if normalized:
        mm,MM = -1.2,1.2
    else:
        mm,MM = min(colors),max(colors)
    logdifs = np.array(colors)
    colors -= mm
    colors /= (MM-mm)
    
    for i in range(N):
        plt.plot([xxx[i],xxx[i+1]],[yyy[i],yyy[i+1]],color = matplotlib.cm.jet(colors[i]),linewidth=2)
    fg = plt.scatter(x,y,c=logdifs)
    plt.scatter(xlamps,ylamps, s=20*pp)
    cbar = plt.colorbar(fg)
    yt = cbar.ax.get_yticks()
    cbar.ax.set_yticklabels(['Ides * {0:.2f}'.format(yi) if yi>=1 else 'Ides / {0:.2f}'.format(1./yi) for yi in exp((MM-mm)*yt+mm)])
    
    II = A*pp
    print 'des. Intensity: '+str(Ides)
    print 'min  Intensity: '+str(min(II))
    print 'max  Intensity: '+str(max(II))
    print 'ratio Imax/Ides:'+str(max(II)/Ides)
    print 'ratio Ides/Imin:'+str(Ides/min(II))
    

Let us now visualize the solution with all lamps on, with a power p=1:

In [ ]:
p_unit = cvx.matrix([1.]*M)
print p_unit
visualize_sol(p_unit)

### Rounded Least-square approximation

In [ ]:
import picos as pic
P = pic.Problem()
p = P.add_variable('p',M)
P.minimize(pic.norm(A*p-Ides)**2,verbose=False)
p_sol = p.value
p_rlsq = cvx.matrix([min(max(pi,0),pmax) for pi in p_sol])

Now, let's visualize the solution !

In [ ]:
visualize_sol(p_rlsq)

### Constrained Least-square approximation (QP)

In [ ]:
#variant 1
import picos as pic
P = pic.Problem()

p = P.add_variable('p',M,lower=0,upper=pmax)
P.minimize(pic.norm(A*p-Ides)**2,verbose=False)

p_lsq = p.value

In [ ]:
#variant 2
import picos as pic
P = pic.Problem()

p = P.add_variable('p',M)

#TODO
P.add_constraint(p >= 0)
P.add_constraint(p <= pmax)

#P.minimize(abs(A*p-Ides)**2,verbose=False)
P.set_objective('min',pic.norm(A*p-Ides)**2)
P.solve(verbose=False)
#TODOend

p_lsq = p.value

In [ ]:
print p_lsq
visualize_sol(p_lsq)

### L1-approximation (LP)

In [ ]:
import picos as pic
P = pic.Problem()
p = P.add_variable('p',M)

#TODO 
P.add_constraint(p >= 0)
P.add_constraint(p <= pmax)
t = P.add_variable('t',1)
P.add_constraint(pic.norm(A*p-Ides,1) <= t)
P.minimize(t,verbose=False)
#TODOend

p_l1 = p.value

In [ ]:
visualize_sol(p_l1)

### Convex-Programming solution

In [ ]:
import picos as pic
P = pic.Problem()
p = P.add_variable('p',M)
I = A*p

#TODO
P.add_constraint(p >= 0)
P.add_constraint(p <= pmax)
u = P.add_variable('u',1)
for k in range(N):
    P.add_constraint(I[k] <= u * Ides)
    P.add_constraint(Ides <= u * I[k])
    
P.minimize(u,verbose=False)
#TODOend

p_cvx = p.value

In [ ]:
visualize_sol(p_cvx)

### LP solution + bisection

In [ ]:
import picos as pic



alpha_lo = 1
alpha_up = 10

alpha = (alpha_lo+alpha_up)/2.

while alpha_up-alpha_lo > 0.0001:
    P = pic.Problem()
    p = P.add_variable('p',M)

    #TODO
    I = A*p
    P.add_constraint( p >= 0 )
    P.add_constraint( p <= pmax )
    
    P.add_constraint( alpha*I >= Ides )
    P.add_constraint( I <= alpha*Ides )
    
    P.minimize( 1|p, verbose=False )
    #TODOend
    
    if P.status == 'optimal':
        alpha_up = alpha
        p_lpb = p.value
    else:
        alpha_lo = alpha
        
    alpha = (alpha_lo+alpha_up)/2.
    print alpha_lo,alpha_up  

In [ ]:
visualize_sol(p_lpb)